In [1]:
import pandas as pd
from thunderpack import ThunderReader
from tqdm import tqdm
import os
import re
import gc

### Discharge Summaries
 - Read in all note metadata CSV files and filter InpatientNoteTypeDSC column for only 'Discharge Summary'
 - Get a list of unique patient ID numbers within this (len = 69684)
 - Randomize the rows of the comprehensive dataframe (len = 539579)
 - Filter for the first instance of each patient ID (essentially a random discharge summary from that patient)
 - Keep track of: Note title, patient ID, date

In [2]:
# year = 2013
# dfs = []
# for i in range(0, 12):
#     df = pd.read_csv(f'/home/jsearle/bigDrive/Dropbox/zz_EHR_Thunderpacks/MGB/MGB_Deidentified_Notes_March12th2024/mgb_notes_{year + i}_metadata.csv')
#     df = df[df['InpatientNoteTypeDSC'] == 'Discharge Summary']
#     dfs.append(df)

output_dir = '/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cohortExtractionPipeline/MGB/CSVs'

# Concatenate all filtered CSV files
all_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.startswith('discharge_summaries_')]
discharge_summaries = pd.concat((pd.read_csv(f) for f in all_files), axis=0, ignore_index=True)

# Show number of total discharge summaries
print(len(discharge_summaries))

# Show example
discharge_summaries.head()

539579


,BDSPPatientID,ContactDate,InpatientNoteTypeDSC,DeidentifiedName
0,121552394,20190106,Discharge Summary,Notes_13407429969_1897463845_20190106.txt
1,121552394,20190106,Discharge Summary,Notes_13407429969_1897464004_20190106.txt
2,121210730,20190105,Discharge Summary,Notes_13338301602_1897145379_20190105.txt
3,118988577,20190114,Discharge Summary,Notes_13351931736_1915990662_20190114.txt
4,116506911,20190118,Discharge Summary,Notes_13339873460_1914565554_20190118.txt


In [3]:
discharged_patients = set(discharge_summaries['BDSPPatientID'])
print(len(discharged_patients))

69684


### Randomization
 - Setting seed (25) just to be able to reproduce results

In [4]:
rand_DS_comp = discharge_summaries.sample(frac=1, random_state=25).reset_index(drop=True)
rand_DS_comp.head()

,BDSPPatientID,ContactDate,InpatientNoteTypeDSC,DeidentifiedName
0,115620551,20150304,Discharge Summary,Notes_13177882086_440619766_20150304.txt
1,114670821,20140924,Discharge Summary,Notes_13211093280_453355769_20140924.txt
2,112187363,20150303,Discharge Summary,Notes_13275669540_440764122_20150303.txt
3,117169043,20141206,Discharge Summary,Notes_13201865993_476529782_20141206.txt
4,122816355,20160824,Discharge Summary,Notes_13404964868_1469502913_20160824.txt


 - Keep only the first instance of each patient ID (Should equal 69684)

In [5]:
rand_DS = rand_DS_comp.drop_duplicates(subset='BDSPPatientID', keep='first')
print(len(rand_DS))

69684


### Combine with ICD table
 - I will be looking at all ICD codes within the month leading up to the date of the discharge summary
 - Check the format of the dates
 - Merge based on patient ID and date

In [7]:
# reader = ThunderReader('/home/jsearle/bigDrive/Dropbox/zz_EHR_Thunderpacks/MGB/thunderpack_icd_9_10_1m_MGB')
# key_length = len(list(reader.keys()))
# print(key_length)

511


In [6]:
# dfs = []
# for i in tqdm(range(1, key_length + 1)):
#     df = reader[f'ICD_partition_{i}']
#     df = df[df['BDSPPatientID'].isin(discharged_patients)]
#     dfs.append(df)

output_dir = '/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cohortExtractionPipeline/MGB/CSVs'

all_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.startswith('filtered_ICD_partition_')]
df = pd.concat((pd.read_csv(f) for f in all_files), axis=0, ignore_index=True)
print(len(df))

27129594


27129594


In [7]:
print(df['ShiftedContactDTS'][2])
print(type(df['ShiftedContactDTS'][2]))

2018-09-30 00:00:00.0000000
<class 'str'>


 - Looks like the Contact Date column is full of strings
 - Converting to TimeStamp will make filtering easier

In [8]:
df['ShiftedContactDTS'] = pd.to_datetime(df['ShiftedContactDTS'])
print(df['ShiftedContactDTS'][2])
print(type(df['ShiftedContactDTS'][2]))

2018-09-30 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [9]:
rand_DS.loc[:, 'ContactDate'] = pd.to_datetime(rand_DS['ContactDate'], format='%Y%m%d')
print(rand_DS['ContactDate'][2])
print(type(rand_DS['ContactDate'][2]))

2015-03-03 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [10]:
merged_df = pd.merge(df, rand_DS, on='BDSPPatientID')

In [11]:
merged_df.head()

,BDSPEncounterID,EncounterLineNBR,BDSPPatientID,ShiftedContactDTS,ICDLineNBR,ICDCD,ICDDSC,DiagnosisNM,DiagnosisDSC,PrimaryDiagnosisFLG,DiagnosisChronicFLG,ShiftedUpdateDTS,DiagnosisLinkedProblemID,BDSPLastModifiedDTS,code_type,ContactDate,InpatientNoteTypeDSC,DeidentifiedName
0,1.362650e+10,1,118474377.0,2023-05-20,1.0,157.1,Malignant neoplasm of body of pancreas,Malignant neoplasm of body of pancreas,NaN,Y,N,2023-05-20 13:25:00.0000000,135026244.0,2023-08-16 02:47:04.0620000,ICD,2019-11-26 00:00:00,Discharge Summary,Notes_13385975835_2644414416_20191126.txt
1,1.358150e+10,3,122336922.0,2022-08-06,1.0,788.99,Other symptoms involving urinary system,Lower urinary tract symptoms,NaN,N,N,2022-08-08 03:51:00.0000000,96060426.0,2022-04-27 13:40:53.7930000,ICD,2023-12-05 00:00:00,Discharge Summary,Notes_13705144828_9349774420_20231205.txt
2,1.342783e+10,11,120522435.0,2018-09-30,1.0,278.01,Morbid obesity,Class 2 severe obesity due to excess calories ...,NaN,N,N,2019-01-09 10:12:00.0000000,33794057.0,2022-04-27 14:57:52.3070000,ICD,2015-12-16 00:00:00,Discharge Summary,Notes_13310872420_756262157_20151216.txt
3,1.335468e+10,3,115980947.0,2018-05-10,1.0,309.81,Posttraumatic stress disorder,Posttraumatic stress disorder,NaN,N,N,2024-01-20 19:05:00.0000000,18320222.0,2023-08-15 19:01:32.7250000,ICD,2016-04-10 00:00:00,Discharge Summary,Notes_13254167247_450575706_20160410.txt
4,1.372510e+10,1,114241241.0,2023-01-20,1.0,465.9,Acute upper respiratory infections of unspecif...,Viral URI,NaN,Y,N,2023-01-20 11:45:00.0000000,NaN,2023-08-16 04:41:57.7350000,ICD,2015-02-04 00:00:00,Discharge Summary,Notes_13264279068_433538463_20150204.txt


At this point I have merged the dataframes based on the patient ID
 - Both dates are listed (ICD date is 'ShiftedContactDTS' while note date is 'ContactDate')
 - Next step is to keep only the rows where the ContactDate is no more than one month after the ShiftedContactDTS

In [13]:
# merged_df_one_month = merged_df[(merged_df['ShiftedContactDTS'] >= merged_df['ContactDate'] - pd.DateOffset(months=1)) & 
#                                 (merged_df['ShiftedContactDTS'] <= merged_df['ContactDate'])]

# +/- one month
merged_df_one_month = merged_df[(merged_df['ShiftedContactDTS'] >= merged_df['ContactDate'] - pd.DateOffset(months=1)) & 
                                (merged_df['ShiftedContactDTS'] <= merged_df['ContactDate'] + pd.DateOffset(months=1))]


In [14]:
print(len(merged_df_one_month))
merged_df_one_month.head()

883350


,BDSPEncounterID,EncounterLineNBR,BDSPPatientID,ShiftedContactDTS,ICDLineNBR,ICDCD,ICDDSC,DiagnosisNM,DiagnosisDSC,PrimaryDiagnosisFLG,DiagnosisChronicFLG,ShiftedUpdateDTS,DiagnosisLinkedProblemID,BDSPLastModifiedDTS,code_type,ContactDate,InpatientNoteTypeDSC,DeidentifiedName
31,1.354543e+10,1,121173304.0,2021-03-07,1.0,205.00,"Acute myeloid leukemia, without mention of hav...",Acute myeloid leukemia not having achieved rem...,NaN,Y,N,2021-03-07 10:46:00.0000000,102807270.0,2022-04-27 13:21:20.6900000,ICD,2021-02-26 00:00:00,Discharge Summary,Notes_13542494271_5936347980_20210226.txt
54,1.349853e+10,1,112711779.0,2022-04-01,1.0,793.11,Solitary pulmonary nodule,"Incidental lung nodule, greater than or equal ...",NaN,N,N,2022-04-01 16:17:00.0000000,104147327.0,2022-04-27 12:58:36.0030000,ICD,2022-04-05 00:00:00,Discharge Summary,Notes_13500753060_5772195018_20220405.txt
81,1.334040e+10,1,117553376.0,2017-09-19,1.0,368.8,Other specified visual disturbances,Blurry vision,NaN,Y,N,2023-11-24 19:37:00.0000000,NaN,2023-08-15 17:26:56.3380000,ICD,2017-08-24 00:00:00,Discharge Summary,Notes_13336483339_1359110280_20170824.txt
180,1.341648e+10,3,111813998.0,2019-01-22,1.0,V49.89,Other specified conditions influencing health ...,Impaired mobility and ADLs,NaN,N,N,2023-11-18 19:01:00.0000000,NaN,2023-08-15 22:56:43.1080000,ICD,2019-02-07 00:00:00,Discharge Summary,Notes_13416480748_2022890910_20190207.txt
192,1.329129e+10,2,116523553.0,2017-04-19,1.0,648.83,"Abnormal glucose tolerance of mother, antepart...",Insulin controlled gestational diabetes mellit...,NaN,N,N,2022-08-17 17:34:00.0000000,NaN,2022-12-24 08:48:29.6970000,ICD,2017-04-25 00:00:00,Discharge Summary,Notes_13291229443_1699851441_20170425.txt


This df is getting a bit hectic
 - Let's remove the fluff and change some column names to be more readable
 - Columns to keep: BDSPPatientID, ShiftedContactDTS, ICDCD, code_type, ContactDate, DeidentifiedName
 - Rename these to: ID, ICD_Date, ICD, CodeType, NoteDate, NoteTitle
 - Move the dates next to each other

In [15]:
columns_to_keep = ['BDSPPatientID', 'ShiftedContactDTS', 'ICDCD', 'code_type', 'ContactDate', 'DeidentifiedName']
clean_df = merged_df_one_month[columns_to_keep]
clean_df.head()

,BDSPPatientID,ShiftedContactDTS,ICDCD,code_type,ContactDate,DeidentifiedName
31,121173304.0,2021-03-07,205.00,ICD,2021-02-26 00:00:00,Notes_13542494271_5936347980_20210226.txt
54,112711779.0,2022-04-01,793.11,ICD,2022-04-05 00:00:00,Notes_13500753060_5772195018_20220405.txt
81,117553376.0,2017-09-19,368.8,ICD,2017-08-24 00:00:00,Notes_13336483339_1359110280_20170824.txt
180,111813998.0,2019-01-22,V49.89,ICD,2019-02-07 00:00:00,Notes_13416480748_2022890910_20190207.txt
192,116523553.0,2017-04-19,648.83,ICD,2017-04-25 00:00:00,Notes_13291229443_1699851441_20170425.txt


In [16]:
rename_dict = {
    'ShiftedContactDTS': 'ICD_Date', 
    'ICDCD': 'ICD', 
    'code_type': 'CodeType', 
    'ContactDate': 'NoteDate', 
    'DeidentifiedName': 'NoteTitle'
}

clean_df = clean_df.rename(columns=rename_dict)
clean_df.head()

,BDSPPatientID,ICD_Date,ICD,CodeType,NoteDate,NoteTitle
31,121173304.0,2021-03-07,205.00,ICD,2021-02-26 00:00:00,Notes_13542494271_5936347980_20210226.txt
54,112711779.0,2022-04-01,793.11,ICD,2022-04-05 00:00:00,Notes_13500753060_5772195018_20220405.txt
81,117553376.0,2017-09-19,368.8,ICD,2017-08-24 00:00:00,Notes_13336483339_1359110280_20170824.txt
180,111813998.0,2019-01-22,V49.89,ICD,2019-02-07 00:00:00,Notes_13416480748_2022890910_20190207.txt
192,116523553.0,2017-04-19,648.83,ICD,2017-04-25 00:00:00,Notes_13291229443_1699851441_20170425.txt


In [18]:
new_order = ['BDSPPatientID', 'ICD_Date', 'NoteDate', 'ICD', 'CodeType', 'NoteTitle']
clean_df = clean_df[new_order]
clean_df.head()

,BDSPPatientID,ICD_Date,NoteDate,ICD,CodeType,NoteTitle
31,121173304.0,2021-03-07,2021-02-26 00:00:00,205.00,ICD,Notes_13542494271_5936347980_20210226.txt
54,112711779.0,2022-04-01,2022-04-05 00:00:00,793.11,ICD,Notes_13500753060_5772195018_20220405.txt
81,117553376.0,2017-09-19,2017-08-24 00:00:00,368.8,ICD,Notes_13336483339_1359110280_20170824.txt
180,111813998.0,2019-01-22,2019-02-07 00:00:00,V49.89,ICD,Notes_13416480748_2022890910_20190207.txt
192,116523553.0,2017-04-19,2017-04-25 00:00:00,648.83,ICD,Notes_13291229443_1699851441_20170425.txt


In [19]:
print(len(clean_df))

883350


Now that the data has been processed a fair amount, lets store the df as a csv that we can just load up directly into another notebook (such as to filter based on cpt code association)

In [20]:
clean_df.to_csv('pre_cpt_filtering.csv', index=False, header=True, sep=',', na_rep='NA')

In [21]:
test = pd.read_csv('pre_cpt_filtering.csv')
test.head()

,BDSPPatientID,ICD_Date,NoteDate,ICD,CodeType,NoteTitle
0,121173304.0,2021-03-07,2021-02-26 00:00:00,205.00,ICD,Notes_13542494271_5936347980_20210226.txt
1,112711779.0,2022-04-01,2022-04-05 00:00:00,793.11,ICD,Notes_13500753060_5772195018_20220405.txt
2,117553376.0,2017-09-19,2017-08-24 00:00:00,368.8,ICD,Notes_13336483339_1359110280_20170824.txt
3,111813998.0,2019-01-22,2019-02-07 00:00:00,V49.89,ICD,Notes_13416480748_2022890910_20190207.txt
4,116523553.0,2017-04-19,2017-04-25 00:00:00,648.83,ICD,Notes_13291229443_1699851441_20170425.txt
